# User Lifecycle plots
Plotting user lifecycle and acculturation gap for subreddits

In [16]:
import sys
import os
# sys.path.append('/homes/gws/taugust/ARK/community_guidelines/lib/')

# do it this way so you have all the functions without SLM_building.[....]
%run -n lib/SLM_building.py
os.chdir('/homes/gws/taugust/ARK/community_guidelines/')
MONTHS = [str(m) for m in range(1,13)]
os.getcwd()


directory already ../data/test_subs_2017/


'/homes/gws/taugust/ARK/community_guidelines'

In [3]:
# subs = ['Naruto', 'pics', 'BabyBumps', 'science', 'politics', 'Cooking']
subs = ['science', 'politics']
slms_subs = {}
# build SLMs only for final month
for i, s in enumerate(subs):
    df_comments, df_author_counts_train = import_csvs(s, path='../cleaned/train/2017/')
    slms_subs[s] = build_monthly_SLM(df_comments, df_author_counts_train, 5)
    
    _, df_author_counts_test = import_csvs(s, path='../cleaned/test/2017/', ext='_test_2017.csv')

    

Importing  ../cleaned/sub_comments/science_comments_2017.csv ..... 

/homes/gws/taugust/miniconda3/envs/reddit/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (0,4,9,10,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


Done
Importing  ../cleaned/train/2017/author_counts/science_author_counts_train_2017.csv ..... Done
Creating  5 SLMs  for  month 1 ..... Done
Creating  5 SLMs  for  month 2 ..... Done
Creating  5 SLMs  for  month 3 ..... Done
Creating  5 SLMs  for  month 4 ..... Done
Creating  5 SLMs  for  month 5 ..... Done
Creating  5 SLMs  for  month 6 ..... Done
Creating  5 SLMs  for  month 7 ..... Done
Creating  5 SLMs  for  month 8 ..... Done
Creating  5 SLMs  for  month 9 ..... Done
Creating  5 SLMs  for  month 10 ..... Done
Creating  5 SLMs  for  month 11 ..... Done
Creating  5 SLMs  for  month 12 ..... Done
Importing  ../cleaned/sub_comments/science_comments_2017.csv ..... Done
Importing  ../cleaned/test/2017/author_counts/science_author_counts_test_2017.csv ..... Done
Importing  ../cleaned/sub_comments/politics_comments_2017.csv ..... 

/homes/gws/taugust/miniconda3/envs/reddit/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (0,1,3,4,5,7,8,9,10,11,12,13,14,15,16,18,19,22) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


Done
Importing  ../cleaned/train/2017/author_counts/politics_author_counts_train_2017.csv ..... Done
Creating  5 SLMs  for  month 1 ..... Done
Creating  5 SLMs  for  month 2 ..... Done
Creating  5 SLMs  for  month 3 ..... Done
Creating  5 SLMs  for  month 4 ..... Done
Creating  5 SLMs  for  month 5 ..... Done
Creating  5 SLMs  for  month 6 ..... Done
Creating  5 SLMs  for  month 7 ..... Done
Creating  5 SLMs  for  month 8 ..... Done
Creating  5 SLMs  for  month 9 ..... Done
Creating  5 SLMs  for  month 10 ..... Done
Creating  5 SLMs  for  month 11 ..... Done
Creating  5 SLMs  for  month 12 ..... Done
Importing  ../cleaned/sub_comments/politics_comments_2017.csv ..... Done
Importing  ../cleaned/test/2017/author_counts/politics_author_counts_test_2017.csv ..... Done


In [12]:
#########################################
# Showing how user cross entropy reduces 
# as authors continue to post in a community. 
#########################################

# Take all users with 1+ posts over every month (long term users) from test author counts
def active_through_months(author_row, months=[str(m) for m in range(1,13)], threshold=5):    
    return author_row[months].apply(lambda x: x > threshold).all()

def get_long_term_users(author_df, threshold=5, num_authors=None):
    df_long_term_authors = author_df[author_df.apply(lambda row: active_through_months(row, threshold=threshold) , axis=1)]
    if num_authors:
        return df_long_term_authors.sample(num_authors)
    else:
        return df_long_term_authors
  

# utility function for getting all text from a single user give a df
# the df will either be from comments or posts,
# text_col allowes for indexing in either (default is comments' 'body')
# NOTE: THIS DOES NOT MODIFY THE BODY TEXT
def get_single_user_text_df(df, author, num_posts=None, text_col='body'):
    df_author = df[df['author'] == author]
    if num_posts:
        return df_author.sample(num_posts)
    else: 
        return df_author 
        

# utility function for getting all the text from a list of users given a df
def get_all_user_text_df(df, authors, num_posts=None, text_col='body'):
    df_author = df[df['author'].apply(lambda x: x in authors)]
    if num_posts:
        sampled_comments = pd.Dataframe()
        df_grouped = df_author.groupby('author')
        for a, g in df_grouped:
            sample = g.sample(num_posts)
            sampled_comments.append(sample)
        return sampled_comments
    else:   
        return df_author




In [24]:
def get_long_term_comments(df_comments, authors, num_posts=None):
    long_term_comments = {a:{} for a in authors}
    for a in authors:
        for m in MONTHS:
            long_term_comments[a][m] = get_user_comments(df_comments, authors=a, month=m, num_posts=num_posts)
    return long_term_comments

def get_long_term_acc_gap(authors, long_term_comments, slms):
    long_term_acc_gap= {a:{} for a in authors}
    for a in authors:
        for m in range(1,13):
            long_term_acc_gap[a][m] = np.mean(calc_month_acc_gap(slms, m, long_term_comments[a][m]))
    return long_term_acc_gap
    
def get_long_term_user_acc_gap(df_comments, df_author_counts, slms, threshold=1):
    long_term_authors = get_long_term_users(df_author_counts, threshold=threshold)
    authors = list(long_term_authors['author'])
    long_term_comments = get_long_term_comments(df_comments,authors)
    long_term_acc_gap = get_long_term_acc_gap(authors,long_term_comments_science, slms_subs['science'])
    return long_term_acc_gap

In [52]:
def calc_entropy_single_text(slms_month_dict, month, text):
    slms = slms_month_dict[month]
    text_bigrams = list(ngrams(text, 2, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>'))
    entropies = []
    for slm in slms:
        slm_entropy = check_ent(slm, text_bigrams)
        entropies.append(slm_entropy)
    return entropies

In [84]:
def plot_long_term_entropy(s, slms_subs): 
    df_comments, df_author_counts_test = import_csvs(s, path='../cleaned/test/2017/', ext='_test_2017.csv')
    threshold = 5
    print('Getting long term authors with more than', colored(str(threshold)+ ' post(s) ', 'red'), '.....', end='')
    long_term_authors = get_long_term_users(df_author_counts_test, threshold=threshold)
    print('Done')
    print('Getting all user text from these ', colored(str(len(long_term_authors))+ ' authors ', 'green'), '....', end='')
    df_long_term_user_text = get_all_user_text_df(df_comments, list(long_term_authors['author']))
    print('Done')
    long_term_user_text_grouped = df_long_term_user_text.groupby('author')
    entropies_df_list = []
    for a, group in long_term_user_text_grouped:
        group['text'] = group['body'].apply(lambda x: [w.lower() for w in word_tokenize(x)[:10]])
        group['entropy'] = group.apply(lambda x:np.mean(calc_entropy_single_text(slms_subs[s],x['created_month'],x['text'])), axis=1)
        mean_entropy_over_month = pd.DataFrame(group.groupby(['created_month'])['entropy'].mean())
        mean_entropy_over_month = mean_entropy_over_month.reset_index()
        entropies_df_list.append(mean_entropy_over_month)

    return entropies_df_list 

In [ ]:
df_list = plot_long_term_entropy('politics', slms_subs)

Importing  ../cleaned/sub_comments/politics_comments_2017.csv ..... Done
Importing  ../cleaned/test/2017/author_counts/politics_author_counts_test_2017.csv ..... Done
Getting long term authors with more than 5 post(s)  .....Done
Getting all user text from these  882 authors  ....Done


/homes/gws/taugust/miniconda3/envs/reddit/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/homes/gws/taugust/miniconda3/envs/reddit/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [79]:
# long_term_user_text_grouped = df_long_term_user_text.groupby('author')
entropies = pd.DataFrame(columns=['created_month', 'entropy'])
for a, group in long_term_user_text_grouped:
    group['text'] = group['body'].apply(lambda x: [w.lower() for w in word_tokenize(x)[:10]])
    group['entropy'] = group.apply(lambda x:np.mean(calc_entropy_single_text(slms_subs['science'],x['created_month'],x['text'])), axis=1)
    mean_entropy_over_month = pd.DataFrame(group.groupby(['created_month'])['entropy'].mean())
    mean_entropy_over_month = mean_entropy_over_month.reset_index()
    entropies = entropies.append(mean_entropy_over_month)
    display(entropies)

/homes/gws/taugust/miniconda3/envs/reddit/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/homes/gws/taugust/miniconda3/envs/reddit/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,created_month,entropy
0,1.0,9.928035
1,2.0,8.664303
2,3.0,10.075780
3,4.0,9.693828
4,5.0,10.381504
5,6.0,10.161422
6,7.0,9.894340
7,8.0,10.049441
8,9.0,9.412704
9,10.0,10.366308


,created_month,entropy
0,1.0,9.928035
1,2.0,8.664303
2,3.0,10.075780
3,4.0,9.693828
4,5.0,10.381504
5,6.0,10.161422
6,7.0,9.894340
7,8.0,10.049441
8,9.0,9.412704
9,10.0,10.366308


,created_month,entropy
0,1.0,9.926711
1,2.0,9.578402
2,3.0,9.875482
3,4.0,9.555723
4,5.0,9.918281
5,6.0,9.733381
6,7.0,9.241108
7,8.0,9.994446
8,9.0,9.541078
9,10.0,9.645633


,created_month,entropy
0,1.0,9.928035
1,2.0,8.664303
2,3.0,10.075780
3,4.0,9.693828
4,5.0,10.381504
5,6.0,10.161422
6,7.0,9.894340
7,8.0,10.049441
8,9.0,9.412704
9,10.0,10.366308


...

,created_month,entropy
0,1.0,10.057765
1,2.0,10.138810
2,3.0,10.130378
3,4.0,9.959372
4,5.0,10.182859
5,6.0,9.904069
6,7.0,9.887024
7,8.0,10.254814
8,9.0,9.665391
9,10.0,9.977715


,created_month,entropy
0,1.0,9.928035
1,2.0,8.664303
2,3.0,10.075780
3,4.0,9.693828
4,5.0,10.381504
5,6.0,10.161422
6,7.0,9.894340
7,8.0,10.049441
8,9.0,9.412704
9,10.0,10.366308


,created_month,entropy
0,1.0,9.609364
1,2.0,8.693321
2,3.0,9.811749
3,4.0,10.171303
4,5.0,9.991879
5,6.0,9.757742
6,7.0,9.138558
7,8.0,9.307986
8,9.0,9.539036
9,10.0,9.160705


,created_month,entropy
0,1.0,9.928035
1,2.0,8.664303
2,3.0,10.075780
3,4.0,9.693828
4,5.0,10.381504
5,6.0,10.161422
6,7.0,9.894340
7,8.0,10.049441
8,9.0,9.412704
9,10.0,10.366308


KeyboardInterrupt: 

In [17]:
# Compare all long term users posts to calculated SLMs for the month that post was made. 
long_term_authors = get_long_term_users(df_author_counts_test, threshold=1)
authors = list(long_term_authors['author'])
long_term_authors_text_df = get_all_user_text_df(df_comments, authors=authors,num_posts=None)

In [25]:
long_term_authors = get_long_term_users(df_author_counts_test, threshold=1)
authors = list(long_term_authors['author'])
long_term_comments_science = get_long_term_comments(df_comments,authors)
long_term_acc_gap_science = get_long_term_acc_gap(authors,long_term_comments_science, slms_subs['science'])

In [30]:
long_term_acc_gap_science

In [29]:
plt.figure(figsize=(10,5))

for a in long_term_acc_gap_science.keys():
    ax = sns.lineplot(x=list(long_term_acc_gap_science[a].keys()), y=list(long_term_acc_gap_science[a].values()))
    
ax.set_title('Acculturation Gap mean for user lifespan(1 yr)', fontsize=10)
ax.set_xlabel('Month', fontsize=10)
ax.set_ylabel('Gap')
# ax.legend(subs_acc.keys(), loc='upper left')

AttributeError: 'NoneType' object has no attribute 'keys'

<Figure size 720x360 with 0 Axes>

In [84]:
mean_long_term_acc_gap = {}
for m in long_term_acc_gap.keys():
    mean_long_term_acc_gap[m] = np.mean(list(flatten(long_term_acc_gap[m])))

In [85]:
mean_long_term_acc_gap

{1: 10.204556305236203,
 2: 10.18921082718725,
 3: 10.305279262757997,
 4: 10.05479615424395,
 5: 10.534492211471155,
 6: 10.469472050549498,
 7: 10.104515067952685,
 8: 10.519249015823204,
 9: 10.265058221016767,
 10: 10.303100304327984,
 11: 10.461112515143833,
 12: 10.549412609241438}

In [ ]:
subs = ['Naruto', 'pics', 'BabyBumps', 'science', 'politics', 'Cooking']

fig, axs = plt.subplots(1, len(subs), sharey=True, tight_layout=True)
fig.set_figheight(10)
fig.set_figwidth(50)
subs_acc = {}
for i, s in enumerate(subs):
    print('-----------------------------------------------')
    print('Acculturation Gap for ', colored(s, 'magenta'))
    print('-----------------------------------------------')

    df_comments, df_author_counts_train = import_csvs(s)
    _, df_author_counts_test = import_csvs(s, path='cleaned/test/2017/', ext='_test_2017.csv')
    
    slms = build_monthly_SLM(df_comments, df_author_counts_train, 1)
    
    monthly_acc_gaps = calc_acc_gap(slms, df_author_counts_test, df_comments)
    
    subs_acc[s] = monthly_acc_gaps
    
    sns.lineplot(x=list(monthly_acc_gaps.keys()), y=[x['gap'] for x in monthly_acc_gaps.values()], ax=axs[i])
    axs[i].set_title('Acculturation Gap for ' + s, fontsize=30)
    axs[i].set_xlabel('Month', fontsize=30)
    axs[i].set_ylabel('Gap (Higher = more different)')
    axs[i].tick_params(axis='both', which='major', labelsize=25)
    
# fig.savefig('../tmp/naruto_acc_gap')